# MCPResource in Dana

This tutorial covers the Model Context Protocol (MCP) resource in Dana, which provides a standardized way to integrate external services and tools into your agents. MCP resources are particularly useful in both the planning and reasoning layers for interacting with external systems.

## Learning Objectives

By the end of this tutorial, you will understand:

1. What MCP is and how it works in the 2-layer architecture
2. How to create and use MCP resources
3. How to work with different transport types (STDIO and HTTP)
4. How to discover and use MCP tools
5. Best practices for MCP resource usage in planning and reasoning

## Prerequisites

- Understanding of Dana's 2-layer architecture
- Familiarity with resource management in Dana
- Basic knowledge of HTTP and STDIO communication

## 1. Understanding MCP in the 2-Layer Architecture

MCP (Model Context Protocol) provides a standardized way to integrate external services into both the planning and reasoning layers. It allows:

- Planning layer to discover and plan using external tools
- Reasoning layer to execute and interact with external services
- Standardized communication between layers and external systems

Let's start with a basic example:

In [ ]:
from dana import MCPResource, ExecutionContext, Plan, PlanStep, ChainOfThoughtStrategy

# Create an MCP resource
mcp_resource = MCPResource(
    name="example_mcp",
    description="Example MCP resource for demonstration",
    transport_type="http",  # or "stdio"
    endpoint="http://localhost:8000"  # for HTTP transport
)

# Create execution context
context = ExecutionContext()

# Register the MCP resource
context.register_resource(mcp_resource)

# Create a plan that uses the MCP resource
plan = Plan(
    name="mcp_plan",
    description="Plan using MCP resource",
    objective="Execute operations using MCP resource"
)

# Add plan steps that use the MCP resource
plan.add_step(PlanStep(
    name="discover_tools",
    description="Discover available MCP tools",
    tool="mcp_discover",
    tool_params={"resource": "example_mcp"}
))

# Create reasoning strategy
reasoning_strategy = ChainOfThoughtStrategy()

# Execute the plan
result = context.execute_plan(plan, reasoning_strategy)

print("Plan Execution Result:")
print(f"- Status: {result.status}")
print(f"- Completed Steps: {result.completed_steps}")
print(f"- Total Duration: {result.total_duration}")

## 2. Working with Different Transport Types

MCP supports two main transport types: HTTP and STDIO. Let's explore both:

In [ ]:
from dana import HTTPTransport, STDIOTransport

# Create HTTP transport MCP resource
http_mcp = MCPResource(
    name="http_mcp",
    description="MCP resource using HTTP transport",
    transport_type="http",
    endpoint="http://localhost:8000",
    transport=HTTPTransport(
        base_url="http://localhost:8000",
        timeout=30,
        retry_count=3
    )
)

# Create STDIO transport MCP resource
stdio_mcp = MCPResource(
    name="stdio_mcp",
    description="MCP resource using STDIO transport",
    transport_type="stdio",
    transport=STDIOTransport(
        command=["python", "mcp_service.py"],
        working_dir="./services"
    )
)

# Example of using both transports in a plan
plan = Plan(
    name="multi_transport_plan",
    description="Plan using multiple transport types",
    objective="Execute operations using different MCP transports"
)

# Add steps for HTTP transport
plan.add_step(PlanStep(
    name="http_operation",
    description="Execute HTTP MCP operation",
    tool="mcp_execute",
    tool_params={
        "resource": "http_mcp",
        "operation": "analyze_data",
        "params": {"data": [1, 2, 3, 4, 5]}
    }
))

# Add steps for STDIO transport
plan.add_step(PlanStep(
    name="stdio_operation",
    description="Execute STDIO MCP operation",
    tool="mcp_execute",
    tool_params={
        "resource": "stdio_mcp",
        "operation": "process_data",
        "params": {"input": "test_data"}
    }
))

## 3. Tool Discovery and Usage

MCP resources can discover and use tools dynamically. Let's see how:

In [ ]:
from dana import ToolDiscovery

# Create a tool discovery instance
discovery = ToolDiscovery()

# Discover tools from an MCP resource
tools = discovery.discover_tools(http_mcp)

print("Discovered Tools:")
for tool in tools:
    print(f"- {tool['name']}: {tool['description']}")
    print(f"  Parameters: {tool['parameters']}")

# Use discovered tools in a plan
plan = Plan(
    name="discovered_tools_plan",
    description="Plan using discovered tools",
    objective="Execute operations using discovered MCP tools"
)

for tool in tools:
    plan.add_step(PlanStep(
        name=f"use_{tool['name']}",
        description=f"Use discovered tool {tool['name']}",
        tool="mcp_execute",
        tool_params={
            "resource": "http_mcp",
            "operation": tool['name'],
            "params": {}
        }
    ))

## 4. Error Handling and Best Practices

Let's look at error handling and best practices for MCP resources:

In [ ]:
from dana import MCPError

class RobustMCPResource(MCPResource):
    """Example of a robust MCP resource implementation."""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._retry_count = kwargs.get('retry_count', 3)
        self._timeout = kwargs.get('timeout', 30)
    
    async def execute_operation(self, operation: str, params: dict) -> dict:
        """Execute an MCP operation with robust error handling."""
        try:
            # Validate operation
            if not self._is_valid_operation(operation):
                raise MCPError(f"Invalid operation: {operation}")
                
            # Validate parameters
            self._validate_params(operation, params)
            
            # Execute with retries
            for attempt in range(self._retry_count):
                try:
                    result = await self._execute_with_timeout(operation, params)
                    return result
                except TimeoutError:
                    if attempt == self._retry_count - 1:
                        raise MCPError(f"Operation timed out after {self._retry_count} attempts")
                    continue
                except Exception as e:
                    if attempt == self._retry_count - 1:
                        raise MCPError(f"Operation failed: {str(e)}")
                    continue
                    
        except MCPError as e:
            # Log error and re-raise
            self._log_error(f"MCP Error: {str(e)}")
            raise
        except Exception as e:
            # Handle unexpected errors
            self._log_error(f"Unexpected error: {str(e)}")
            raise MCPError(f"Unexpected error: {str(e)}")
    
    def _is_valid_operation(self, operation: str) -> bool:
        """Check if operation is valid."""
        return operation in self._get_available_operations()
    
    def _validate_params(self, operation: str, params: dict) -> None:
        """Validate operation parameters."""
        required_params = self._get_required_params(operation)
        for param in required_params:
            if param not in params:
                raise MCPError(f"Missing required parameter: {param}")
    
    def _log_error(self, message: str) -> None:
        """Log error message."""
        print(f"[ERROR] {message}")

## 5. Testing MCP Resources

Let's see how to test MCP resources:

In [ ]:
import pytest

# Test data
TEST_OPERATION = "test_operation"
TEST_PARAMS = {"param1": "value1", "param2": "value2"}

# Test the MCP resource
async def test_mcp_resource():
    resource = RobustMCPResource(
        name="test_mcp",
        description="Test MCP resource",
        transport_type="http",
        endpoint="http://localhost:8000",
        retry_count=2,
        timeout=10
    )
    
    # Test valid operation
    result = await resource.execute_operation(TEST_OPERATION, TEST_PARAMS)
    assert "result" in result
    
    # Test invalid operation
    with pytest.raises(MCPError):
        await resource.execute_operation("invalid_operation", {})
        
    # Test missing parameters
    with pytest.raises(MCPError):
        await resource.execute_operation(TEST_OPERATION, {})

# Run the tests
if __name__ == "__main__":
    pytest.main([__file__])

## Next Steps

In this tutorial, we've covered:

1. Understanding MCP in the 2-layer architecture
2. Working with different transport types
3. Tool discovery and usage
4. Error handling and best practices
5. Testing MCP resources

In the next tutorial, we'll explore the MCP (Model Control Protocol) resource in more detail, focusing on advanced features and integration patterns.